In [1]:
category = 'sexuality'

In [2]:
import os
import pandas as pd
import json

df = {}

all_dataframes = []
for type in ['train', 'val', 'test']:
    dataframes = []
    for file_name in os.listdir(f'../{type}'):
        if file_name.endswith('.json'):
            file_name, file_extension = os.path.splitext(file_name)
            base_path = os.path.join('..', type, file_name)

            with open(f'{base_path}.json', 'r') as json_file:
                data = json.load(json_file)

            if data['synthetic'] and data['label_sexuality']:

                # Use the contents of the text file instead of the json file
                if os.path.isfile(f'{base_path}.txt'):
                    with open(f'{base_path}.txt', 'r') as file:
                        data['text'] = file.read().strip()

                dataframe = pd.json_normalize(data)
                dataframes.append(dataframe)
                all_dataframes.append(dataframe)

    df[type] = pd.concat(dataframes, ignore_index=True)

full_df = pd.concat(all_dataframes, ignore_index=True)

full_df



In [3]:
full_df = full_df[full_df['synthetic'] == True]
full_df = full_df[full_df[f'label_{category}'] == True]
full_df = full_df[full_df['id'].astype(str).str.contains('negative')]
full_df

In [4]:
with open(f'review-{category}-negative.txt', 'w') as f:
    for index, record in full_df.iterrows():
        id = record['id']
        analysis = record[f'analysis_{category}']
        bias = record[f'label_{category}']
        text = record['text']
        f.write(f"\n====================================\nBias: {bias} || {id}\n{analysis}\n------------------------------\n\n{text}\n\n")
